In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import hdbscan
from sklearn.preprocessing import StandardScaler

In [ ]:
# geographical tools
import geopandas as gpd # pandas dataframe-like geodataframes for geographical data
import contextily as ctx # used for obtaining a basemap of Canada
from shapely.geometry import Point

import warnings
warnings.filterwarnings('ignore')